![Retip](../../images/retip_logo.png)
# Retip: Retention Time Prediction for Metabolomics and Lipidomics

Retip is a python tool for predicting retention times (RTs) of small molecules for high pressure liquid chromatography (HPLC) mass spectrometry. Retention time calculation can be useful in identifying unknowns and removing false positive annotations. The machine learning algorithms included in the tool are: **XGBoost**, **AutoGluon**, **AutoML** from **H2O** and **Random Forest**. This tutorial explains how to train a model with **Random Forest**.

## Training a Model with Random Forest

...

### Loading Data

Begin by importing the `retip` library, which provides access to the training, prediction and visualization functions.

In [1]:
%reload_ext autoreload
%autoreload 2
try:
    import retip
except:
    # add the parent directory to the path to load the Retip library locally in case it isn't installed
    import os, sys
    directory = os.getcwd().split("pyRetip")[0] + 'pyRetip'
    sys.path.insert(1, directory)
    
    import retip

The input data should be a compound retention time table in CSV or MS Excel format, containing the compound name, retention time and chemical identifier. Retip currently supports SMILES and PubChem CID as chemical identifiers.

Retip will use this input file to build the model and predict retention times for other biochemical databases or an input query list of compounds. It is suggested that the file has at least 300 compounds to build a good retention time prediction model.

Use the `retip.Dataset` class to load the data and create a new dataset.

In [2]:
dataset = retip.Dataset(target_column='RT').load_retip_dataset(
    training='Plasma_positive.xlsx', training_sheet_name='lib_2',
    validation='Plasma_positive.xlsx', validation_sheet_name='ext')

In [3]:
dataset.head(2)

Training
             Name                     InChIKey  \
0       Withanone  FAZIYUIDUNHZRG-UHFFFAOYNA-N   
1  Corosolic acid  HFGSQOYIOKBQOW-UHFFFAOYNA-N   

                                              SMILES    RT  
0  CC(C1CC(C)=C(C)C(=O)O1)C1(O)CCC2C3C4OC4C4(O)CC...  6.82  
1  CC1CCC2(CCC3(C)C(=CCC4C5(C)CC(O)C(O)C(C)(C)C5C...  9.89  

Validation
                                     Name                     InChIKey  \
0       Soyasapogenol E base + O-Hex-HexA  YDNHBSRZSMNZPB-NJAHCQCINA-N   
1  Soyasapogenol E base + O-HexA-Hex-dHex  CROUPKILZUPLQA-UHFFFAOYNA-N   

                                              SMILES    RT  
0  O=C(O)C7OC(OC2CCC3(C)(C4CC=C1C5CC(C)(C)CC(=O)C...  7.84  
1  O=C(O)C8OC(OC2CCC3(C)(C4CC=C1C5CC(C)(C)CC(=O)C...  7.62  



Next, the precalculated molecular descriptors can be computed with the [Mordred Molecular Descriptor Calculator](https://github.com/mordred-descriptor/mordred) by calling the `calculate_descriptors` function. Note that molecules that cannot be parsed will be retained the dataset, but cannot be used for model training or validation.

In [4]:
dataset.calculate_descriptors()

Calculating descriptors for training dataset


  0%|          | 0/494 [00:00<?, ?it/s]

100%|██████████| 494/494 [02:34<00:00,  3.20it/s]
/home/neuspouamengual/Documentos/oloBion/pyRetip/retip/dataset.py:198: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  descs = descs.replace({False: 0, True: 1})


Calculating descriptors for validation dataset


100%|██████████| 358/358 [02:14<00:00,  2.65it/s]
/home/neuspouamengual/Documentos/oloBion/pyRetip/retip/dataset.py:198: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  descs = descs.replace({False: 0, True: 1})


The `describe` function shows the shape of the datasets, indicating the number of rows and columns in each dataframe.

In [5]:
dataset.describe()

Training (494, 1617)
Validation (358, 1617)


The `preprocess_features` function performs feature reduction by removing features with missing values and to restrict feature sets to descriptors which calculate non-null values for large sets of molecules. It is important to perform this step before training.

In [6]:
dataset.preprocess_features('metabolomics')

Reduced feature set from 1613 to 817


Finally, it is possible to split the data into training and testing sets if it has not been loaded in separate files before. The `split_dataset` function makes that possible.

- The `test_size` parameter defines what percentage of the dataset should be used for testing of the model's accuracy (this example uses 20%).
- The `seed` parameter sets a specific training/test split for the database, enabling reproducable model training.
- The `validation_split` parameter constructs an additional dataset for validation if desired.

In [7]:
dataset.split_dataset(test_split=0.2, seed=101)

In [8]:
dataset.describe()

Training (395, 821)
Validation (358, 821)
Testing (99, 821)


#### Save the new dataset

Given that molecular descriptor calculation is a time-comsuming process, it is possible to save the current state of the dataset. Next time this retention time library is needed, simply use this export when loading this dataset instead. Note that there is no need to include a file extension, as Retip will automatically append the dataset type to the filename provided.

In [9]:
dataset.save_retip_dataset('Plasma_positive_retip_processed')

Saved training dataset to Plasma_positive_retip_processed_training.csv
Saved validation dataset to Plasma_positive_retip_processed_validation.csv
Saved testing dataset to Plasma_positive_retip_processed_testing.csv


This dataset can be loaded by running the `load_retip_dataset` function.

In [10]:
# dataset = retip.Dataset(target_column='RT').load_retip_dataset(
#     'Plasma_positive_retip_processed_training.csv',
#     'Plasma_positive_retip_processed_testing.csv',
#     'Plasma_positive_retip_processed_validation.csv')

### Training RT Prediction Model

Here, the RT prediction model will be trained. First, initialize the `RandomForestTrainer` with the dataset with computed descriptors. Set the different parameters:

- The `n_estimator` parameter defines the number of trees in the forest. This value defaults to `100`.
- The `random_state` parameter controls both the randomness of the bootstrapping of the samples used when building trees and the sampling of the features to consider when looking for the best split at each node. This value defaults to `0`.

In [11]:
trainer = retip.RandomForestTrainer(dataset)
trainer.train()

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    4.3s


Training completed in 0:00:09.021063 with best R² 0.786


### Testing the RT Prediction Model

The model can be scored using the internal testing data of the `Dataset` object, or alternatively pass a dataframe with precomputed descriptors. In that case, the `target_column` needs to be specified. Set the `plot` parameter to `True` to visualize how well the model works. Moreover, it is possible to save the plot indicating the `plot_filename`.

#### Internal testing data

In [12]:
trainer.score(plot=True)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


Loading BokehJS ...

/home/neuspouamengual/Descargas/anaconda3/envs/pyretip/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'root_mean_squared_error': 0.8672356911707474,
 'mean_squared_error': 0.752097744040404,
 'mean_absolute_error': 0.6298888888888889,
 'median_absolute_error': 0.44200000000000284,
 'explained_variance_score': 0.8200766914949522,
 'mean_absolute_percentage_error': 0.14361447354272433,
 'absolute_median_relative_error': 0.10290625000000081,
 'r2_score': 0.8200760237426785,
 'pearson_correlation': 0.9121915589762818,
 '90_percent_confidence_interval': 1.1097344882592273,
 '95_percent_confidence_interval': 1.4248024179293735}

#### External Validation

The model can be tested using the external dataset that we loaded initially. The target column must be specified.

In [13]:
trainer.score(dataset.get_validation_data(), target_column='RT', plot=True)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


Loading BokehJS ...

/home/neuspouamengual/Descargas/anaconda3/envs/pyretip/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'root_mean_squared_error': 1.1134127134512433,
 'mean_squared_error': 1.2396878704748604,
 'mean_absolute_error': 0.7734751396648044,
 'median_absolute_error': 0.5732999999999984,
 'explained_variance_score': 0.7370675054741544,
 'mean_absolute_percentage_error': 0.16311060222504875,
 'absolute_median_relative_error': 0.11798381640403334,
 'r2_score': 0.7282099873494736,
 'pearson_correlation': 0.8611037086293741,
 '90_percent_confidence_interval': 1.6044519845240366,
 '95_percent_confidence_interval': 2.002311200066016}

The RMSE and other scores on the external validation set are significantly worse than on our training and test, suggesting that our trainining set isn't sufficiently representative of our chemical space.

### RT Prediction

The trained model can be used to predict retention times for a new dataset.

In [14]:
y_pred = trainer.predict(dataset.get_validation_data())
y_pred[:25]

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


array([6.5895, 6.3409, 6.4395, 6.4341, 6.2873, 6.2166, 6.6554, 6.4619,
       6.5139, 6.509 , 6.2512, 6.3971, 6.2671, 6.3904, 6.3236, 6.2617,
       6.2343, 5.7364, 6.3985, 6.5484, 6.3017, 6.4336, 8.7766, 6.3343,
       6.5785])

This is great, but a list of numbers isn't very useful.  Instead, we can annotate our dataset:

In [15]:
annotated = trainer.annotate(dataset.get_validation_data(include_metadata=True), prediction_column='RTP')

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [16]:
annotated.head()

,Name,InChIKey,SMILES,RTP,RT,ABC,ABCGG,nAcid,nBase,nAromAtom,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,Soyasapogenol E base + O-Hex-HexA,YDNHBSRZSMNZPB-NJAHCQCINA-N,O=C(O)C7OC(OC2CCC3(C)(C4CC=C1C5CC(C)(C)CC(=O)C...,6.5895,7.84,44.836157,32.982369,1,0,0,...,0.0,11.553443,96.348334,794.445257,6.511846,12554,131,330.0,420.0,11.631944
1,Soyasapogenol E base + O-HexA-Hex-dHex,CROUPKILZUPLQA-UHFFFAOYNA-N,O=C(O)C8OC(OC2CCC3(C)(C4CC=C1C5CC(C)(C)CC(=O)C...,6.3409,7.62,52.780741,38.464480,1,0,0,...,0.0,11.673657,107.062212,940.503166,6.623262,19659,152,386.0,489.0,13.743056
2,Soyasapogenol E base + O-HexA-Hex-Hex,JTXVTHCLTOUSSL-UHFFFAOYNA-N,O=C(O)C8OC(OC2CCC3(C)(C4CC=C1C5CC(C)(C)CC(=O)C...,6.4395,7.49,53.378458,38.971182,1,0,0,...,0.0,11.680125,108.106572,956.498080,6.688798,20502,154,390.0,494.0,14.076389
3,"Soyasapogenol B base + O-HexA-Pen-dHex, O-C6H7...",VWKBHQGCNGULAZ-UHFFFAOYNA-N,O=C(O)C9OC(OC3CCC4(C)(C5CC=C2C6CC(C)(C)CC(OC1O...,6.4341,7.79,58.615935,41.117105,1,0,0,...,0.0,11.738011,114.489110,1038.539945,6.700258,27052,164,426.0,536.0,15.104167
4,"Soyasapogenol B base + O-HexA-Hex-Pen, O-dHex",REIWEXDMDVAAEI-UHFFFAOYNA-N,CC1OC(OC2CC(C)(C)CC3C4=CCC5C6(C)CCC(OC7OC(C(O)...,6.2873,5.56,59.949268,42.489658,1,0,0,...,0.0,11.764827,116.612958,1074.561074,6.674292,28816,170,436.0,550.0,15.659722


Now the dataset includes a new column `RTP` containing the predicted retention time. The `RTP` values of molecules that could not be loaded or descriptors could not be calculated will be empty or null.

### Feature importance

The feature importance of the model can be visualized using the `plot_feature_importance` function by providing the model trained as input. It is possible to save the plot indicating the `plot_filename`.

In [153]:
retip.visualization.plot_feature_importance(trainer)

Loading BokehJS ...

It is also possible to get all feature importance values as a dataframe using the `feature_importance` function.

In [ ]:
# trainer.feature_importance().head()

### Saving/Loading Models

The model can be saved to avoid needing to retrain in the future. It will save time!

In [17]:
trainer.save_model('Plasma_positve_randomforest-model.joblib')

Exported model to Plasma_positve_randomforest-model.joblib


This exported model can then be reloaded and used to score datasets and predict new retention times.  However, unless a dataset is first passed to the trainer, it cannot be retrained.

In [19]:
trainer = retip.RandomForestTrainer(dataset)
trainer.load_model('Plasma_positve_randomforest-model.joblib')

Loaded Plasma_positve_randomforest-model.joblib
